# scipy.sparse

https://www.scipy.org/

SciPy предназначена для выполнения научных и инженерных расчётов.

Согласно wiki, основные модули:


| субпакет      | реализованная "функциональность"                                 |
|---------------|------------------------------------------------------------------|
| **constants** |Физические константы и коэффициенты пересчёта (с версии 0.7.0[2]).|
| **cluster **  |Векторное квантование.                                            |
|**fftpack ** |Дискретные алгоритмы преобразования Фурье.                        |
|**integrate** |Инструменты для интегрирования.|
|**interpolate **|Инструменты для интерполяции.|
|**io** |Ввод-вывод данных.|
|**lib** |Работа со сторонними библиотеками.|
|**linalg** |Линейная алгебра.|
|**misc** |Разное.|
|**optimize **|Средства оптимизации.|
|**sandbox** |Экспериментальный код.|
|**signal** |Обработка сигналов.|
|**sparse **|Поддержка разреженных матриц.|
|**special** |Cпециальные функции.|
|**stats** |Статистические функции.|
|**weave **|Использование кода, написанного на языках C и C++.|


## scipy.sparse
### разреженные матрицы
copy-pasted from http://dobroflot.blogspot.ru/2014/11/scipy.html

RTFM: https://docs.scipy.org/doc/scipy-0.18.1/reference/sparse.html

#### bsr_matrix
Этот класс подойдёт для разреженных матриц, в которых ненулевые элементы "идут блоками". Можно указать высоту и ширину блоков; по умолчанию эти параметры подбираются эвристически.

Насколько я понимаю, хорош только для хранения. 

#### coo_matrix
Как пишут в документации, матрицы, представленные в *COOrdinate format*, то есть как тройки (x,y,value).

Не поддерживаются арифметические операции и выделение подматриц, колонок и строк. Если встречаются тройки с одинаковыми координатами, значения суммируются.

Хорош для построения матриц или как промежуточный формат между другими способами представления разреженных матриц (csc, csr).

#### csc_matrix
Элементы хранятся по колонкам. Следовательно, главная фишка -- эффективное выделение колонок (слайсинг).

Кроме того, эффективно реализованы операции CSC + CSC, CSC * CSC и т. д. Сравнительно быстро работает умножение на столбец, но ясно, что в CSR это реализовано эффективнее.

Дороги выделение строк и модификация нулевых элементов.

#### csr_matrix
Элементы хранятся по строкам. Следовательно, главная фишка -- их эффективное выделение (слайсинг).

Кроме того, эффективно реализованы операции CSR + CSR, CSR * CSR и т. д. Быстрое умножение на столбец.

Дороги выделение столбцов и модификация нулевых элементов.


#### dia_matrix
Хитрый формат. Диагонали хранятся как строки матрицы, которую подаём конструктору. Также подаётся массив со "сдвигами" от главной диагонали. Причём каждую следующую мы как бы "приписываем к левому краю матрицы".

Насколько я понимаю, хорош для хранения матриц, у которых много пустых диагоналей.

#### dok_matrix
Словарь, в котором ключ -- пара координат. Быстрый доступ до элементов, быстрое преобразование в COO (скан по всем ключам). Дубликатов не бывает.

Всё остальное -- медленное.


#### lil_matrix
Цепной список цепных списков, представляющих строки матрицы. Эффективный слайсинг строк, эффективное добавление новых элементов.

Арифметика -- медленная, умножение на вектор -- медленное, слайсинг колонок -- медленный.

Хорош для построения. Если матрица ну совсем большая, то лучше использовать COO.

Работают многие методы, уже знакомые нам по занятию и домашке по NumPy.
Например, hstack, vstack, ...

Задач на **scipy.sparse** не будет, но, возможно, этот модуль вам пригодится в домашней работе.

# pandas 
-- мощный инструмент с не самым очевидным API

http://pandas.pydata.org/

http://pandas.pydata.org/pandas-docs/version/0.18.1/10min.html

http://pandas.pydata.org/pandas-docs/version/0.18.1/tutorials.html

Exploratory data analysis, предобработка данных, predictive modeling (в малой степени).
Проекции, слияние, фильтрация, группировка, агрегация,...

Есть смысл читать документацию и думать о наибольшей эффективности выполнения тех или иных операций.
Так, по-прежнему, если вы пишете цикл, вы что-то делаете не так; векторные операции for the win!

И особенно это важно, если ваш код будет затем адаптироваться для [py]Spark. Благодаря DataFrame API Apache Spark, это делается просто.

## data structures and types    

In [1]:
import pandas as pd
import numpy as np

# series -- "колонка" одного типа; можно относиться как к словарю из ИНДЕКСОВ в ЗНАЧЕНИЯ
s = pd.Series([1,3,5,np.nan,6,8], name="Some series")

print(s.dtype)
print(s.shape)
print(s.index)

s[2:4]

float64
(6,)
RangeIndex(start=0, stop=6, step=1)


2    5.0
3    NaN
Name: Some series, dtype: float64

In [2]:
s = pd.Series([1,3,5,np.nan,6,8], name="Some series", index=["q","w","e","r","t","y"])

print(s.index)

print(s[2:4])

print(s["e":"t"]) # обратите внимание!

print(s["e":"q"]) # обратите внимание!


Index(['q', 'w', 'e', 'r', 't', 'y'], dtype='object')
e    5.0
r    NaN
Name: Some series, dtype: float64
e    5.0
r    NaN
t    6.0
Name: Some series, dtype: float64
Series([], Name: Some series, dtype: float64)


In [4]:
# http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
dates = pd.date_range('20130101', periods=6) # default frequency measure is 1 day

print(dates)

df = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))

print(df)

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')
                   A         B         C         D
2013-01-01  0.537812 -1.341104 -0.347894 -1.384417
2013-01-02 -0.675753  1.141779  0.287904 -0.465170
2013-01-03 -0.488427  0.597972  2.087767  1.963745
2013-01-04 -0.852172 -0.050369 -0.176193  0.221053
2013-01-05 -0.685809  0.148580 -0.372406 -0.928716
2013-01-06 -1.201493  0.536798  1.065499  1.187133


In [5]:
df2 = pd.DataFrame({ 'A' : 1.,
                    'B' : pd.Timestamp('20130102'),
                    'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
                    'D' : np.array([3] * 4,dtype='int32'),
                    'E' : pd.Categorical(["test","train","test","train"]),
                    'F' : 'foo' })
df2

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo


In [6]:
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
dtype: object

## viewing data

In [7]:
df.head(1), df.tail(2)

(                   A         B         C         D
 2013-01-01 -0.325481  0.564172  0.691345  0.348965,
                    A         B         C         D
 2013-01-05  0.185373 -0.868162 -1.176157  0.517679
 2013-01-06 -1.227492  1.487468 -1.795868 -0.144867)

In [8]:
print(df.index)
#print(df.columns)
#print(df.values)

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')


In [9]:
# стандартные статистики по каждой колонке
df.describe()

,A,B,C,D
count,6.000000,6.000000,6.000000,6.000000
mean,-0.282953,0.356866,-0.475116,-0.028289
std,0.829022,0.882769,1.127584,0.502341
min,-1.227492,-0.868162,-1.795868,-0.905205
25%,-0.821878,-0.250176,-1.361268,-0.140614
50%,-0.398433,0.617186,-0.546811,0.006849
75%,0.057659,0.774407,0.539143,0.297112
max,1.079975,1.487468,0.770417,0.517679


In [10]:
df.T 

,2013-01-01 00:00:00,2013-01-02 00:00:00,2013-01-03 00:00:00,2013-01-04 00:00:00,2013-01-05 00:00:00,2013-01-06 00:00:00
A,-0.325481,1.079975,-0.938710,-0.471385,0.185373,-1.227492
B,0.564172,0.809142,-0.521625,0.670200,-0.868162,1.487468
C,0.691345,0.770417,-1.422971,0.082535,-1.176157,-1.795868
D,0.348965,-0.127855,0.141553,-0.905205,0.517679,-0.144867


In [27]:
df.T.index, df.index

(Index(['A', 'B', 'C', 'D'], dtype='object'),
 DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
                '2013-01-05', '2013-01-06'],
               dtype='datetime64[ns]', freq='D'))

In [11]:
df.sort_index(axis=1, ascending=False), df

(                   D         C         B         A
 2013-01-01  0.348965  0.691345  0.564172 -0.325481
 2013-01-02 -0.127855  0.770417  0.809142  1.079975
 2013-01-03  0.141553 -1.422971 -0.521625 -0.938710
 2013-01-04 -0.905205  0.082535  0.670200 -0.471385
 2013-01-05  0.517679 -1.176157 -0.868162  0.185373
 2013-01-06 -0.144867 -1.795868  1.487468 -1.227492,
                    A         B         C         D
 2013-01-01 -0.325481  0.564172  0.691345  0.348965
 2013-01-02  1.079975  0.809142  0.770417 -0.127855
 2013-01-03 -0.938710 -0.521625 -1.422971  0.141553
 2013-01-04 -0.471385  0.670200  0.082535 -0.905205
 2013-01-05  0.185373 -0.868162 -1.176157  0.517679
 2013-01-06 -1.227492  1.487468 -1.795868 -0.144867)

In [12]:
df.sort_values(by='B')

,A,B,C,D
2013-01-05,0.185373,-0.868162,-1.176157,0.517679
2013-01-03,-0.938710,-0.521625,-1.422971,0.141553
2013-01-01,-0.325481,0.564172,0.691345,0.348965
2013-01-04,-0.471385,0.670200,0.082535,-0.905205
2013-01-02,1.079975,0.809142,0.770417,-0.127855
2013-01-06,-1.227492,1.487468,-1.795868,-0.144867


## выборка данных
#### NB!
While standard Python / Numpy expressions for selecting and setting are intuitive and come in handy for interactive work, for production code, we recommend the optimized pandas data access methods, .at, .iat, .loc, .iloc and .ix.
http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing
http://pandas.pydata.org/pandas-docs/stable/advanced.html#advanced

In [13]:
df['A']

2013-01-01   -0.325481
2013-01-02    1.079975
2013-01-03   -0.938710
2013-01-04   -0.471385
2013-01-05    0.185373
2013-01-06   -1.227492
Freq: D, Name: A, dtype: float64

In [14]:
df.A

2013-01-01   -0.325481
2013-01-02    1.079975
2013-01-03   -0.938710
2013-01-04   -0.471385
2013-01-05    0.185373
2013-01-06   -1.227492
Freq: D, Name: A, dtype: float64

In [17]:
df[0:3]

,A,B,C,D
2013-01-01,-0.325481,0.564172,0.691345,0.348965
2013-01-02,1.079975,0.809142,0.770417,-0.127855
2013-01-03,-0.938710,-0.521625,-1.422971,0.141553


## выборка по значениям индексов

In [18]:
 df['20130102':'20130104']

,A,B,C,D
2013-01-02,1.079975,0.809142,0.770417,-0.127855
2013-01-03,-0.938710,-0.521625,-1.422971,0.141553
2013-01-04,-0.471385,0.670200,0.082535,-0.905205


In [9]:
# первый индекс
print("First index: " + str(dates[0]))

print(df)

# выбираем всё по этому индексу
print(df.loc[dates[0]])

# при слайсинге включаются обе границы

First index: 2013-01-01 00:00:00
                   A         B         C         D
2013-01-01  0.537812 -1.341104 -0.347894 -1.384417
2013-01-02 -0.675753  1.141779  0.287904 -0.465170
2013-01-03 -0.488427  0.597972  2.087767  1.963745
2013-01-04 -0.852172 -0.050369 -0.176193  0.221053
2013-01-05 -0.685809  0.148580 -0.372406 -0.928716
2013-01-06 -1.201493  0.536798  1.065499  1.187133
A    0.537812
B   -1.341104
C   -0.347894
D   -1.384417
Name: 2013-01-01 00:00:00, dtype: float64


## выборка по порядковым номерам индексов

In [24]:
df.iloc[3:5]

,A,B,C,D
2013-01-04,-0.471385,0.670200,0.082535,-0.905205
2013-01-05,0.185373,-0.868162,-1.176157,0.517679


In [25]:
# доступ до отдельного значения
print(df.iat[1, 1])
print(df.iloc[1, 1]) # то же

0.80914202776
0.80914202776


In [26]:
# выборка по значениям в колонках [ничего не напоминает?]
df[df.A > 0]

,A,B,C,D
2013-01-02,1.079975,0.809142,0.770417,-0.127855
2013-01-05,0.185373,-0.868162,-1.176157,0.517679


In [27]:
# фильтрация
df2 = df.copy()
df2['E'] = ['one', 'one','two','three','four','three']

# то же, что filter по вхождению
df2[df2['E'].isin(['hello', 'woow', 'three', 'one'])]

,A,B,C,D,E
2013-01-01,-0.325481,0.564172,0.691345,0.348965,one
2013-01-02,1.079975,0.809142,0.770417,-0.127855,one
2013-01-04,-0.471385,0.670200,0.082535,-0.905205,three
2013-01-06,-1.227492,1.487468,-1.795868,-0.144867,three


## обновление

In [29]:
s1 = pd.Series([1,2,3,4,5,6], index=pd.date_range('20130102', periods=6))
df['F'] = s1
df

,A,B,C,D,F
2013-01-01,-0.325481,0.564172,0.691345,0.348965,NaN
2013-01-02,1.079975,0.809142,0.770417,-0.127855,1.0
2013-01-03,-0.938710,-0.521625,-1.422971,0.141553,2.0
2013-01-04,-0.471385,0.670200,0.082535,-0.905205,3.0
2013-01-05,0.185373,-0.868162,-1.176157,0.517679,4.0
2013-01-06,-1.227492,1.487468,-1.795868,-0.144867,5.0


In [32]:
# выборка по индексам

df.at[:,'A'] = 0
df.iat[0,1] = 999999
df.loc[:,'D'] = np.array([5] * len(df))
df

,A,B,C,D,F
2013-01-01,0,0.564172,0.691345,0.348965,NaN
2013-01-02,0,0.809142,0.770417,-0.127855,1.0
2013-01-03,0,-0.521625,-1.422971,0.141553,2.0
2013-01-04,0,0.670200,0.082535,-0.905205,3.0
2013-01-05,0,-0.868162,-1.176157,0.517679,4.0
2013-01-06,0,1.487468,-1.795868,-0.144867,5.0


In [42]:
# выборка по условию
df[df > 0] = - 0.001 * df
df

,A,B,C,D,F
2013-01-01,0,-999.999000,-0.000691,-0.005,NaN
2013-01-02,0,-0.000809,-0.000770,-0.005,-0.001
2013-01-03,0,-0.521625,-1.422971,-0.005,-0.002
2013-01-04,0,-0.000670,-0.000083,-0.005,-0.003
2013-01-05,0,-0.868162,-1.176157,-0.005,-0.004
2013-01-06,0,-0.001487,-1.795868,-0.005,-0.005


In [49]:
# Задачи
# https://www.kaggle.com/c/titanic/data

import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Задание 0
# поисследуйте распределения значений фич -- 
# info, describe, etc...

print(train.head(5))
# видим, что есть пустые значения
# google: dropna, fillna

# Задание 1
# Отбросить колонку Cabin и другие колонки, которые нам точно не помогут предсказать, погибнет ли пассажир.



   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [ ]:
# Задание 2
# Написать код, заполняющий пропуски в "численных" колонках 
# 2a -- значением (-1)
# 2b -- средним значением по колонке

In [ ]:

# Задание 3
# Понять, какие фичи -- номинальные (категориальные) -- без осмысленного порядка над ними. 
# Применить к ним пандасовский dummy_encoding (google examples)


In [ ]:

# Задание 4
# Выделить Survived как отдельный вектор (или Series)


In [ ]:
# Задание 5
# Нормализуйте средствами pandas (нельзя использовать sklearn) отдельно 
# каждую колонку -- с возрастом и со стоимостью билета
# 5a. min-max scaling
# 5b. вычесть среднее и разделить на стандартное отклонение

In [ ]:
# Задание 6
# Постройте
# train.pivot_table('PassengerId', 'Pclass', 'Survived', 'count').plot(kind='bar', stacked=True)
# Погуглите, что это? О чём нам говорит этот график?

In [ ]:
# Задание 7
# Для каждой фичи, кажущейся вам полезной, постройте гистограмму с помощью pandas-hist

In [ ]:
# Задание 8
# Обучите slkearn-овскую LogisticRegression по преобразованному вами train 
# (в котором уже не должно остаться строковых колонок и NaN). Проверьте качество на test.csv.
#
# Проделайте то же самое без нормализации численных фич. Кто победил?

In [ ]:
# Задание 9*
# Поработайте над кодом, над данными train и логистической регрессией так, чтобы
# 1) параметры логистической регрессии подбирались с помощью GridSearch,
# 2) качество оценивалось на train с помощью KFold-валидации.

### На самостоятельное  изучение

- можно читать/писать в экселевские файлы
- можно работать с временными рядами [time sehdp5ries]
- можно строить графики "из коробки"
- можно отбрасывать дубликаты
- *можно грабить корованы...*